In [23]:
import re
import json
import string
import pandas as pd

from datasets import load_from_disk, Dataset, DatasetDict

test_dataset = load_from_disk("./data/test_dataset/")
test_df = pd.DataFrame(test_dataset['validation'])

with open("./data/wikipedia_documents.json", 'r', encoding='utf-8') as f:
    wiki_json = json.load(f)
wiki_df = pd.DataFrame(list(wiki_json.values()))


In [24]:
def flattenList(nested_list):
    flat_list = []
    for sublist in nested_list:
        for item in sublist:
            flat_list.append(item)
    return flat_list

special_characters = re.escape(string.punctuation)

def getOtherCharacters(text):
    text = re.sub(r"[가-힣ㄱ-ㅎㅏ-ㅣA-Za-z0-9一-龥ぁ-ゔァ-ヴー々〆〤 ]", "", text)
    text = re.sub(f'[{special_characters}]', "", text)
    text = re.sub(r"[≪≫《》〈〉＜＞「」『』‘’“”・·°∧­ćä]", "", text)
    text = re.sub(r"[\*\#]+", "",text) # 특수기호 '*', '#'  
    return text

other_characters = [getOtherCharacters(it) for it in wiki_df['text']]
other_characters = set(flattenList(other_characters))
other_characters = (''.join(other_characters))

In [25]:
def removeOtherCharacters(row):
    question = test_df['question'][row]

    question = re.sub(f'[{other_characters}]', "", question)
    question = question.replace('\\n', ' ').replace('\n', ' ')
    question = ' '.join(question.split())

    test_df.loc[row, 'question'] = question

In [26]:
for row in test_df.itertuples():
    i = row.Index
    removeOtherCharacters(i)

In [27]:
test_dataset = Dataset.from_dict(test_df)
dataset = DatasetDict({"validation":test_dataset})

dataset.save_to_disk("./data/test_preprocessed/")

Saving the dataset (1/1 shards): 100%|██████████| 600/600 [00:00<00:00, 156038.10 examples/s]
